In [1]:
import numpy as np
import torch
import time

In [2]:
def potential(r,k,l,alpha_list,scale=100):
    result=0
    for i in range(len(alpha_list)):
        result+=alpha_list[i]*r**(i-1)
        # 采用球bessel函数之类的“势能基”会不会更好?
    return result*scale-k*(l+1)*l/r**2  # 离心势能是固定的

In [3]:
h_bar=1
m=1
b_lap:float=-h_bar**2/(2*m)

# 同时对于库伦势函数, 取e=1, 4\pi\epsilon_0=scale, E_n=-1/(2n^2)
dtype=torch.float64
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

La=0
Lb =10
L=Lb-La  # domain length
N = 2048   # number of interior points # 对时间成本来说几乎是平方量级
h :float= L / (N+1)

# 控制势函数的大小
scale=100

In [4]:
batch=16
epoch=2
eig_num=12
l_max=24
para_num=5
width=torch.tensor([4,0.1,0.01,0.01,0.01],dtype=dtype,device=device)

grid=torch.linspace(La,Lb,N+2,dtype=dtype,device=device)
grid=grid[1:-1]


# Construct the tridiagonal matrix A
diag = -2.0 / h**2 * torch.ones(N,device=device) * b_lap
off_diag = 1.0 / h**2 * torch.ones(N - 1,device=device) * b_lap


alpha_list_list=[]
eig_list_list=[]
for _ in range(batch):
    alpha_list=2*width*torch.rand(para_num,device=device)-width
    V_diag=potential(grid,b_lap,0,alpha_list,scale)
    A_list=[]

    # 保证势能的大小合适,不至于截断失效或者精度过低
    while V_diag[0]<2*torch.mean(torch.abs(V_diag)) or V_diag[0]>100*torch.mean(torch.abs(V_diag)):
        alpha_list=2*width*torch.rand(para_num,device=device)-width
        V_diag=potential(grid,b_lap,0,alpha_list,scale)
    
    A = torch.diag(diag) + torch.diag(off_diag,diagonal=1) + torch.diag(off_diag, diagonal=-1)+torch.diag(V_diag)
    A_list.append(A)
    for l in range(1,l_max):
        V_diag=potential(grid,b_lap,l,alpha_list,scale)
        A = torch.diag(diag) + torch.diag(off_diag,diagonal=1) + torch.diag(off_diag, diagonal=-1)+torch.diag(V_diag)
        A_list.append(A)
    A_list=torch.stack(A_list,dim=0)
    eigenvalues= torch.linalg.eigvalsh(A_list)
    eigenvalues=eigenvalues[:,0:eig_num]
    eigenvalues=eigenvalues.reshape(-1)
    eig_min_pinch,_=torch.topk(eigenvalues,k=eig_num,largest=False)
    alpha_list_list.append(alpha_list)
    eig_list_list.append(eig_min_pinch)
alpha_list_list=torch.stack(alpha_list_list,dim=0)
eig_list_list=torch.stack(eig_list_list,dim=0)


In [5]:
print("eigenvalues:",eig_list_list)
print("alpha_list:",alpha_list_list)

eigenvalues: tensor([[  37.9698,   38.1838,   38.6054,   39.2237,   40.0241,   40.9907,
           42.1076,   42.4523,   42.6858,   43.1446,   43.3596,   43.8141],
        [  74.4141,   74.5630,   74.8595,   75.3012,   75.8845,   76.6050,
           77.4577,   78.4369,   79.5367,   80.1117,   80.2678,   80.5786],
        [   7.2131,    7.4276,    7.8373,    8.4139,    9.1290,    9.7641,
            9.9589,   10.0170,   10.4916,   10.8844,   11.1461,   11.8910],
        [  15.3985,   15.6446,   16.1231,   16.8112,   17.6831,   18.7141,
           19.1198,   19.3999,   19.8821,   19.9395,   20.7063,   21.1687],
        [  80.6250,   80.7701,   81.0591,   81.4899,   82.0594,   82.7637,
           83.5981,   84.5576,   85.6367,   86.4629,   86.6143,   86.8301],
        [   3.2607,    3.4640,    3.8008,    4.2291,    4.4085,    4.7180,
            4.7258,    5.1636,    5.2770,    5.6827,    5.6881,    5.8735],
        [-599.8139, -599.8035, -599.7826, -599.7513, -599.7095, -599.6572,
      